In [249]:
from dataclasses import dataclass
import numpy as np

In [250]:
with open('./input.txt', 'r') as f:
    input = f.read()
    
with open('./test_input.txt', 'r') as f:
    test_input = f.read()

In [251]:
def process_input(input):
    input = [ line.strip() for line in input.split('\n') ]
    draw_numbers = [ int(number) for number in input[0].split(',') ]
    
    index = 0
    boards = [[]]
    for row in input[2:]:
        if row == '':
            index += 1
            boards.append([])
        else:
            boards[index].append([ int(square) for square in filter(lambda x: x.isnumeric(), row.split(' ')) ])
    return draw_numbers, boards

In [252]:
@dataclass
class Square:
    number: int
    marked: bool = False

class Board():
    def __init__(self, board):
        self.board = np.array([ [ Square(square) for square in row ] for row in board ])
        
    def __str__(self):
        return '\n'.join( [' '.join(row) for row in [ [ str(square.number).rjust(2) for square in row ] for row in self.board ]] )
    
    def mark_square(self, number):
        for row in self.board:
            for square in row:
                if square.number == number:
                    square.marked = True
                    break
                    
    def is_winner(self):
        for row in self.board:
            if self._row_is_winner(row):
                return True
        
        for row in self.board.transpose():
            if self._row_is_winner(row):
                return True
            
        return False
    
    def _row_is_winner(self, row):
        return sum( [ 1 if square.marked else 0 for square in row ] ) == 5
    
    def sum_unmarked(self):
        return sum( [ square.number if not square.marked else 0 for row in self.board for square in row ] )

In [253]:
# evaluation conditions
def winning_evaluation(boards):
    for board in boards:
        if board.is_winner():
            return True, board,boards
    return False, None, boards
        
def losing_evaluation(boards):
    if len(boards) == 1 and boards[0].is_winner():
        return True, boards[0], boards
    remaining_boards = list(filter(lambda board: not board.is_winner(), boards))
    return False, None, remaining_boards

In [254]:
def bingo(input, evaluation):
    
    def final_score(board, number):
        return board.sum_unmarked() * number

    def bingo_round(boards, number, evaluation):
        for board in boards:
            board.mark_square(number)
        return evaluation(boards)
    
    draw_numbers, boards = process_input(input)
    boards = [ Board(board) for board in boards ]
    for number in draw_numbers:
        game_over, best_board, boards = bingo_round(boards, number, evaluation)
        if game_over:
            return final_score(best_board, number)

In [255]:
# Part 1
bingo(input, winning_evaluation)

74320

In [256]:
# Part 2
bingo(input, losing_evaluation)

17884